# 2024-2 URP assignment

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
import re
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time

In [2]:
# 데이터프레임 선언
Q_df = pd.DataFrame(columns = ['문서 번호', '제목',  '질문',  "위치 고유번호" , '조회수', '질문 날짜'])
A_df = pd.DataFrame(columns = ['문서 번호', '답변 ID',  '답변', '답변 날짜'])
loc_dict = pd.read_csv("naver_kin_regions_df_l2.csv", index_col = "지역 코드")

# URL에서 dirID 값을 가져오는 함수

In [3]:
def get_dirid(url):
    match = re.search(r'dirId=(\d+)', url)
    dirid_value = match.group(1)
    if len(dirid_value) >= 6 :
        dirid_value = dirid_value[:6]
    return dirid_value

# URL에서 docID 값을 가져오는 함수

In [4]:
def get_docid(url):
    match = re.search(r'docId=(\d+)', url)
    docid_value = match.group(1)
    return docid_value

# 반복되는 패턴을 통해서 docID를 통해 위치 반환하는 함수(추후 매핑 테이블을 이용할 예정)

In [5]:
# dirID -> loc
def get_loc(id=""):
    url = 'https://kin.naver.com/qna/list.naver?dirId=12'+id

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')

    location = soup.find(attrs={'class': 'location'}).get_text(strip=True)
    return location

In [6]:
def find_pattern(text):
    # 문자열 길이의 절반까지 탐색(더 긴 패턴은 두 번 반복될 수 없으므로)
    for length in range(1, len(text)):
        # 길이 `length`만큼 패턴을 잘라서 비교
        pattern = text[:length]
        # 패턴이 두 번 연속 반복되는지 확인
        if text.startswith(pattern * 2):
            return pattern

In [7]:
# 지역 & 플레이스 기본 정보 문자열 제거를 위한 변수 지정
url2 = 'https://kin.naver.com/qna/list.naver?dirId=12'
headers2 = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response2 = requests.get(url2, headers=headers2)
soup2 = BeautifulSoup(response2.text, 'html.parser')
useless_information = soup2.find(attrs={'class': 'location'}).get_text(strip=True)

def return_location(search):
    a = get_loc(search[:-2]+"01").replace(get_loc(search[:-2]),"")
    a_elim = get_loc(search[:-len(search)]+search[-len(search):-2]).replace(get_loc(search[:-len(search)]),"")
    first_loc = find_pattern(a)
    a = a[len(first_loc):]
    b = get_loc(search).replace(a,"")
    c = b.replace(useless_information, "")
    return c.replace(a_elim,"")

def return_full_location(search):
    full_location = ""
    n = 2
    while n <= len(search):
        full_location += return_location(search[:n])+ " "
        n += 2
    return full_location

# 최종적으로 질문과 답변 정보 받아오는 함수

In [8]:
def collect_information(url, Q_df, A_df):
    # 페이지 요청
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    # BeautifulSoup으로 페이지 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # 질문 제목 추출
    title = soup.find(attrs={'class': 'endTitleSection'}).get_text(strip=True)
    title = title.replace(",","")
    Q_detail= soup.find(attrs={'class': 'questionDetail'}).get_text(strip=True,separator=' ')
    Q_detail = Q_detail.replace(",","")
    Q_info = soup.find_all(class_='infoItem')
    Q_view = Q_info[0].get_text()[3:]
    Q_date = Q_info[1].get_text()[3:]
    if Q_date.endswith(" 전"):
        Q_date = datetime.today().strftime('%Y.%m.%d')
    if Q_date.startswith("작성일"):
        Q_date = Q_date.replace("작성일", "")
    ans = []
    A_dates = []
    try:
        answers = soup.find_all(class_='se-main-container')
        ans_date = soup.find_all(class_='answerDate')
        for answer in answers:
            ans.append(answer.get_text(strip=True).replace('\u200b', ''))
        for ans_dates in ans_date:
            A_dates.append(ans_dates.get_text(strip=True))
    except:
        answer_error = "아직 답변이 없습니다"
    
    ID = get_dirid(url)
    doc_ID = get_docid(url)
    Q_inf_list =[get_docid(url), title, Q_detail, ID, Q_view, Q_date]
    A_inf_list =[]
    n = 1
    for i,j in zip(ans,A_dates):
        changed = i.replace(","," ")
        if j.endswith(" 전"):
            today = datetime.today().strftime('%Y.%m.%d.')
            A_inf_list.append([doc_ID, doc_ID+str(n), changed[2:],today])
        else:
            A_inf_list.append([doc_ID, doc_ID+str(n), changed[2:],j])
        n += 1
    Q_df.loc[Q_df.shape[0]] = Q_inf_list
    
    for i in A_inf_list:
        A_df.loc[A_df.shape[0]] = i
    return Q_df, A_df

# 질문 리스트에서 각 질문의 URL 값 가져오기

In [9]:
# 질문 목록에서 URL만 가져기기
urls = []
def get_url_from_list(search_url):
    global urls
    response = requests.get(search_url) 
    
    soup = BeautifulSoup(response.text, 'html.parser')

    elements = soup.find_all(class_=re.compile(r'^_nclicks:kls_new\.list'))
    
    for element in elements[1:]:
        href = "https://kin.naver.com"+element.get('href')
        urls.append(href)
    return urls

### list 페이지 설정
### 삭제 직전인 데이터는 못불러옴 예시로 보여주는 수 밖에 없다. 글로 설명 불가능

In [10]:
def collect_urls(region):
    try:
        for j in range(15,40):
            for i in range(1,21):
                page = f"https://kin.naver.com/qna/list.naver?dirId=12{region}{j}&queryTime=2024-11-06%2017%3A33%3A26&page={i}"
                urls= get_url_from_list(page)
    except:
        pass
    return urls

In [11]:
urls = collect_urls("02")
urls

['https://kin.naver.com/qna/detail.naver?d1id=12&dirId=12021502&docId=477436354',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120215&docId=477307816',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120215&docId=477302990',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120215&docId=477148433',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120215&docId=477046506',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120215&docId=476944123',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120215&docId=476515085',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120215&docId=476212069',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120215&docId=476019577',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=12021502&docId=475842750',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120215&docId=475627851',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=120215&docId=475610943',
 'https://kin.naver.com/qna/detail.n

In [12]:
len(urls)

150

# 실제 데이터 불러오기(매핑 테이블 추후 SQL에서 join 예정)

In [13]:
for url in tqdm(urls, desc="Processing URLs", ncols=100, leave=True):
    collect_information(url,Q_df, A_df)

Processing URLs: 100%|████████████████████████████████████████████| 150/150 [00:47<00:00,  3.19it/s]


In [14]:
Q_df

,문서 번호,제목,질문,위치 고유번호,조회수,질문 날짜
0,477436354,질문사진속 배경 어딘지 아시는분... 대교에 타워 있는곳.,대교에 타워 있는곳... 여기 어딘지 지역 아시는분??,120215,22,2024.11.03
1,477307816,질문초등학교 운동장에 잔디 깔면 걸리는 시간 좋은점..?,깔면 걸리는 시간 좋은점..나쁜점 알려주세욥,120215,20,2024.10.31
2,477302990,질문버스노선.,부산영도에서 김해조은금강병원까지 버스타고 가는 방법을 부탁드립니다^^,120215,25,2024.10.31
3,477148433,질문남도여중에 대해서,지금 현재 12년생 초 6인데요 부산 영도구 남도여중에 대해서 궁금해서요 혹시 졸업...,120215,17,2024.10.27
4,477046506,질문부산동구 불?,저는 부산 영도구에사는대 왜이리 동구에 불이났는대 영도구소방서가 나서나요? 동구관할...,120215,16,2024.10.24
...,...,...,...,...,...,...
145,476248693,질문부산 기장군 구간단속 질문,부산 롯데월드에서 기장군 정관읍가는 구간단석 질문입니다. 고속도로 구간단속 진입속도...,120218,17,2024.10.05
146,476162355,질문수원 장안 고등학교에 대해 궁금한 것이 있습니다,장안 고등학교에 전형 방법의 구체적인 내용 진학을 위한 준비사항 이 학교에 단점이 ...,120218,19,2024.10.03
147,475826488,질문부산 배달,중학생인데 돈을 모아야 하는 일이 생겨서 그런데 중학생도 배달을 할 수 있나요? (...,120218,34,2024.09.25
148,475081298,질문정관 도서관 주차장,정관 도서관 이용 시간이 끝나도 주차해도 괜찮을까요?,120218,17,2024.09.08


In [15]:
A_df

,문서 번호,답변 ID,답변,답변 날짜
0,477436354,4774363541,대교전라남도 영광군 염산면 옥실리영광칠산타워전라남도 영광군 염산면 향화로 2-10다...,2024.11.03.
1,477302990,4773029901,구청 버스정류장에서 8번 버스 탑승하시고 사상역 정류장에 하차하여바로 근처 김해경전...,2024.10.31.
2,477046506,4770465061,에 소방차가 몇대없어요. 불나면 다른 구에서 지원나갑니다.,2024.10.24.
3,476944123,4769441231,동쪽 충무동에 여인숙이 있습니다 남포동 자갈치 시장에서 계속 우측으로 시장을 따라 ...,2024.10.22.
4,476515085,4765150851,다리에서 불꽃놀이를 즐길 수 있어요.답변이 도움되셨으면 좋겠네요 좋은하루보내세요-올...,2024.10.12.
...,...,...,...,...
153,476782857,4767828572,게도 187번밖에없네요ㅠㅠㅠ187첫차05:00|막차22:00|배차간격50분윗반송역=...,2024.10.18.
154,476729385,4767293851,산숲에서는 택시가 잘 잡히지 않습니다,2024.10.17.
155,476729385,4767293852,부르면 되겠죠.,2024.10.17.
156,476344793,4763447931,자님이 궁금해하는 기장지역 일대에 맛집이 은근히 많습니다이렇게 방송에 출연한 맛집들...,2024.10.08.


In [16]:
Q_df.to_csv('Questions_경기도.csv', index = False)
A_df.to_csv('Answers_경기도.csv', index = False)